In [12]:
import pandas as pd

train_data = pd.read_csv('titanic/train.csv')
test_data = pd.read_csv('titanic/test.csv')

In [13]:
train_data.head()
train_data.info()
train_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699113,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526507,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.416700,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
#fill missing values
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
train_data['Fare'].fillna(train_data['Fare'].median(), inplace=True)

# fill missing test data
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# handle cabin data
train_data['Cabin'] = train_data['Cabin'].fillna('Unknown')
test_data['Cabin'] = train_data['Cabin'].fillna('Unknown')

# create a new feature for the deck which is the first letter of cabin
train_data['Deck'] = train_data['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'U')
test_data['Deck'] = test_data['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'U')

#drop original cabin column
train_data.drop(columns=['Cabin'], inplace=True)
test_data.drop(columns=['Cabin'], inplace=True)


In [15]:
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked','Deck'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked','Deck'], drop_first=True)
# this makes these binary values like isMale, EmbarkedfromQ, EMbarkedfromS,
# etc and then deck1 , deck2, deck? ...


In [16]:
# drop non useful col
train_data.drop(columns=['Name', 'Ticket',], inplace=True)
test_data.drop(columns=['Name', 'Ticket'], inplace=True)


In [17]:
# feature selection

features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S', 
            'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Deck_U']
X_train = train_data[features]
y_train = train_data['Survived']
X_test = test_data[features]

In [18]:
# logisitcal Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# innit and train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_split, y_train_split)

y_pred = model.predict(X_val)
print("Accuracy: ", accuracy_score(y_val, y_pred))


Accuracy:  0.8100558659217877


In [19]:
# # rf I want to see how other methods fare

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# # Initialize and train the Random Forest model
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train_split, y_train_split)

# # Validate the Random Forest model
# y_pred_rf = rf_model.predict(X_val)
# print("Random Forest Validation Accuracy:", accuracy_score(y_val, y_pred_rf))
# 78% accuracy compare to others 81

In [20]:
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': y_test_pred
})
submission.to_csv('titanic_submission.csv', index=False)